In [2]:
import mxnet as mx
import time
import numpy as np
from sklearn.cluster import KMeans
import time

/home/tapir/miniconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def get_quantized_filters(filters, shrink=16):
    shape=filters.shape
    n_clusters=shape[0]/shrink

    filters_shaped=filters.reshape((shape[0], shape[1] * shape[2] * shape[3]))
    estimator = KMeans(n_clusters=n_clusters)
    estimator.fit(filters_shaped.asnumpy())

    filter_kmean_indexes = estimator.predict(X=filters_shaped.asnumpy())
    filters_quantized = np.array([estimator.cluster_centers_[idx] for idx in filter_kmean_indexes])

    return mx.nd.array(filter_kmean_indexes), mx.nd.array(estimator.cluster_centers_), mx.nd.array(filters_quantized)

          
def get_onehot(data, nclusters, batch_size):
    index_mat = mx.nd.one_hot(data, depth=nclusters).reshape(0, -1)
    return mx.nd.broadcast_axes(mx.nd.expand_dims(index_mat, axis=0), axis=0, size=batch_size)

In [17]:
shrink = 2
batch_size=32

img =mx.nd.random.uniform(0, 1, shape=(batch_size,64,224,224))
filters =mx.nd.random.uniform(0, 1, shape=(32,16,3,3))

fshape = filters.shape

nclusters=filters.shape[0]/shrink
indices, codebook_filters, quantized_filters= get_quantized_filters(filters,shrink)
quantized_filters=quantized_filters.reshape(filters.shape)
codebook_filters=codebook_filters.reshape(nclusters,fshape[1],fshape[2], fshape[3])
onehot_indices=get_onehot(indices,nclusters,batch_size)



In [18]:
begin=time.time()
for i in range(100):
    res1=mx.nd.Convolution(img,weight=quantized_filters,kernel=(3,3), num_filter=filters.shape[0], no_bias=True).asnumpy()
print time.time()-begin
oshape=res1.shape

1.13187098503


In [19]:
begin=time.time()
for i in range(100):
    conv2=mx.nd.Convolution(img,weight=codebook_filters,kernel=(3,3), num_filter=codebook_filters.shape[0], no_bias=True)
    res2=mx.nd.batch_dot(onehot_indices,conv2.reshape(0,0,-1)).reshape(oshape).asnumpy()
print time.time()-begin


1.15659189224


In [7]:
np.array_equal(res1.asnumpy(), res2.asnumpy())

True

In [ ]:
indices2[0]

In [ ]:
mx.nd.one_hot(indices, depth=a.shape[1]).shape


In [24]:
def get_onehot(data, nclusters, batch_size):
    index_mat = mx.nd.one_hot(data, depth=nclusters).reshape(0, -1)
    return mx.nd.broadcast_axes(mx.nd.expand_dims(index_mat, axis=0), axis=0, size=batch_size)

In [25]:
a=mx.nd.zeros((1,128,32,32))
indices = mx.nd.array(np.random.choice(128, 256)) #shape = (128,)
indices2= mx.nd.one_hot(indices, depth=a.shape[1])
indices3= get_onehot(indices,a.shape[1], a.shape[0])

In [26]:
begin=time.time()

for i in range(50):
    cfilter0 = a.slice_axis(axis=0, begin=0, end=0+1).reshape(-3,0,0)
    dummy   = mx.nd.take(cfilter0,indices).expand_dims(axis=0)

    for ch in range(1,a.shape[0]):
        cfilter = a.slice_axis(axis=0, begin=ch, end=ch+1).reshape(-3,0,0)
        cfres   = mx.nd.take(cfilter,indices).expand_dims(axis=0)
        dummy   = mx.nd.concat(dummy, cfres, dim=0)
    
    res1=dummy.asnumpy()

print time.time()-begin
print res1.shape
    

0.120637893677
(1, 256, 32, 32)


In [27]:

begin=time.time()

for i in range(50):
    arr=[]

    for ch in range(a.shape[0]):
        cfilter = a.slice_axis(axis=0, begin=ch, end=ch+1).reshape(-3,0,0)
        arr.append(mx.nd.take(cfilter,indices))

    res2 = mx.nd.stack(*arr).asnumpy()  

print time.time()-begin

print res2.shape

0.134425878525
(1, 256, 32, 32)


In [28]:

begin=time.time()

for i in range(50):

    channels=mx.nd.split(data=a,axis=0,num_outputs=a.shape[0],squeeze_axis=True)

    for cidx, ch in enumerate(channels):
        channels[cidx]=mx.nd.take(cfilter,indices)

    res3 = mx.nd.stack(*channels).asnumpy()  

print time.time()-begin

print res3.shape


ValueError: operands could not be broadcast together with remapped shapes[original->remapped]: (256L, 32L, 32L) and requested shape (32L, 32L)

In [29]:

begin=time.time()

for i in range(50):

    res4 = mx.nd.batch_dot(indices3,a.reshape(0,0,-1)).reshape(res3.shape).asnumpy()

print time.time()-begin

print res4.shape


NameError: name 'res3' is not defined

In [10]:

begin=time.time()

for i in range(50):
    
    channels=mx.nd.split(data=a,axis=0,num_outputs=a.shape[0],squeeze_axis=True)

    for cidx, ch in enumerate(channels):
        channels[cidx]=mx.nd.dot(indices2,ch)

    res5 = mx.nd.stack(*channels).asnumpy()  
    
print time.time()-begin

print res5.shape


1.32261896133
(32, 128, 32, 32)


In [11]:

begin=time.time()

for i in range(50):
    
    channels=mx.nd.split(data=a,axis=0,num_outputs=a.shape[0],squeeze_axis=True)

    for cidx, ch in enumerate(channels):
        channels[cidx]=mx.nd.sparse.dot(indices2,ch)

    res6 = mx.nd.stack(*channels).asnumpy()  
    
print time.time()-begin

print res6.shape


1.30165910721
(32, 128, 32, 32)


In [11]:
np.array_equal(res6,res1)

True

In [ ]:
re

In [ ]:
x=mx.nd.zeros((16,32,32))
y=mx.nd.zeros((16,32,32))

mx.nd.stack(x,y).shape

In [ ]:
range(1,5)

In [ ]:
x=mx.nd.zeros((7,16,32,32))
ls=mx.nd.split(data=x,axis=0,num_outputs=7,squeeze_axis=True)

In [ ]:
for l in ls:
    print l.shape

In [37]:
b=mx.nd.zeros((1,128,32,32))
bindices = mx.nd.array(np.random.choice(128, 512)) #shape = (128,)
bindices2= mx.nd.one_hot(bindices, depth=b.shape[1])
bindices3= get_onehot(bindices,b.shape[1], b.shape[0])

In [38]:

begin=time.time()

for i in range(1000):
    
    bres1 = mx.nd.take(b.reshape(-3,0,0),bindices).asnumpy()
    
print time.time()-begin

print bres1.shape


2.3684527874
(512, 32, 32)


In [39]:
print bindices2.shape
begin=time.time()

for i in range(1000):
    
    bres2 = mx.nd.dot(bindices2, b.reshape(-3,0,0)).asnumpy()
    
print time.time()-begin

print bres2.shape


(512L, 128L)
1.81407117844
(512, 32, 32)


In [40]:
print bindices2.shape
begin=time.time()

for i in range(1000):
    
    bres3 = mx.nd.sparse.dot(bindices2, b.reshape(-3,0,0)).asnumpy()
    
print time.time()-begin

print bres3.shape


(512L, 128L)
1.90993189812
(512, 32, 32)


In [41]:
begin=time.time()

for i in range(1000):
    
    bres4 = mx.nd.batch_dot(bindices3, b.reshape(0,0,-1)).asnumpy()
    
print time.time()-begin

print bres4.shape


1.75696587563
(1, 512, 1024)
